In [1]:
# Prepare packages
import numpy as np
import cv2
import matplotlib.pyplot as plt
import idx2numpy
import os

import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

In [2]:
# Define the device to be used for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Prepare data
# Hyperparameters
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# Dataset
train_dataset = torchvision.datasets.CIFAR10(root='../datasets', train=True, 
                                             transform=transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='../datasets', train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, shuffle=False)

In [4]:
# Convolutional neural network (CNN) model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 5 * 5, 1024)
        self.fc2 = nn.Linear(1024, 10)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    

# Define my dataset
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]           2,432
         MaxPool2d-2           [-1, 32, 14, 14]               0
            Conv2d-3           [-1, 64, 10, 10]          51,264
         MaxPool2d-4             [-1, 64, 5, 5]               0
            Linear-5                 [-1, 1024]       1,639,424
           Dropout-6                 [-1, 1024]               0
            Linear-7                   [-1, 10]          10,250
Total params: 1,703,370
Trainable params: 1,703,370
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.32
Params size (MB): 6.50
Estimated Total Size (MB): 6.83
----------------------------------------------------------------


In [7]:
# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/500], Loss: 1.7792
Epoch [1/10], Step [200/500], Loss: 1.6778
Epoch [1/10], Step [300/500], Loss: 1.3412
Epoch [1/10], Step [400/500], Loss: 1.4960
Epoch [1/10], Step [500/500], Loss: 1.2957
Epoch [2/10], Step [100/500], Loss: 1.3095
Epoch [2/10], Step [200/500], Loss: 1.2094
Epoch [2/10], Step [300/500], Loss: 1.3788
Epoch [2/10], Step [400/500], Loss: 1.1217
Epoch [2/10], Step [500/500], Loss: 1.3395
Epoch [3/10], Step [100/500], Loss: 1.0862
Epoch [3/10], Step [200/500], Loss: 1.1393
Epoch [3/10], Step [300/500], Loss: 1.1342
Epoch [3/10], Step [400/500], Loss: 1.0569
Epoch [3/10], Step [500/500], Loss: 1.0244
Epoch [4/10], Step [100/500], Loss: 1.2155
Epoch [4/10], Step [200/500], Loss: 0.9128
Epoch [4/10], Step [300/500], Loss: 0.9683
Epoch [4/10], Step [400/500], Loss: 0.8472
Epoch [4/10], Step [500/500], Loss: 0.9241
Epoch [5/10], Step [100/500], Loss: 1.0001
Epoch [5/10], Step [200/500], Loss: 1.0537
Epoch [5/10], Step [300/500], Loss: 0.9893
Epoch [5/10

In [8]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {:.2f}%'.format(correct * 100 / total))

Test Accuracy of the model on the 10000 test images: 70.48%
